# Signal
_____
**An informal introduction to the Signal protocol**

**References:**
* [[X3DH](https://signal.org/docs/specifications/x3dh/)] The X3DH Key Agreement Protocol, Marlinspike and Perrin, 2016.
* [[Double Ratchet](https://signal.org/docs/specifications/doubleratchet/)] The Double Ratchet Algorithm, Perrin and Marlinspike, 2016.
* [[Sesam](https://signal.org/docs/specifications/sesame/)] The Sesame Algorithm: Session Management for Asynchronous Message Encryption, Marlinspike and Perrin. 2017.
* [[PQXDH](https://signal.org/docs/specifications/pqxdh/) The PQXDH Key Agreement Protocol, Kret and Schmidt, 2023
* [[WhatsApp](https://faq.whatsapp.com/820124435853543)] WhatsApp Encryption Overview: Technical white paper, 2023.
* [[CDGS](https://eprint.iacr.org/2016/1013)] A Formal Security Analysis of the Signal Messaging Protocol, Cohn-Gordon et al. 2020
* [[CCG](https://eprint.iacr.org/2016/221)] On Post-compromise Security, Cohn-Gorden et al. 2016
* [[CPZ](https://eprint.iacr.org/2019/1416.pdf)] The Signal Private Group System and Anonymous Credentials Supporting Efficient Verifiable Encryption, Chase et al. 2020
* [[VGIK](https://eprint.iacr.org/2021/642)] On the Cryptographic Deniability of the Signal Protocol, Vantandas et al. 2021
* [[BFGJS](https://eprint.iacr.org/2021/769)] Post-quantum Asynchronous Deniable Key Exchange and the Signal Handshake, Brendel et al. 2022


**References DMA:**
* [[DMA gatekeeper obligations](https://www.eu-digital-markets-act.com/Digital_Markets_Act_Article_7.html)] Obligation for gatekeepers on interoperability
* [[EU Law](https://eur-lex.europa.eu/eli/reg/2022/1925/oj)] EU DMA law

## Signal Protocol
* Origins date back to [TextSecure](https://en.wikipedia.org/wiki/TextSecure) App started in 2010
    + TextSecure used a *double ratchet* construction referred to as *Axolotl Ratchet* to highlight ''self-healing properties''
<p style="text-align:center;font-size:10px">
<video height="150" autoplay muted loop src="./vid/axolotl.mp4"></video>
</p>
* Later TextSecure was merged with RedPhone into Signal in 2015
    + maintained by a non-profit organization called Open Whisper Systems
* 2018 Open Whisper Systems was transformed into [Signal Foundation](https://signalfoundation.org/)
    + recent podcast with CEO Meredith Whittaker [LNP504](https://logbuch-netzpolitik.de/lnp504-the-politics-of-intellectual-shame)

## Signal Protocol Usage
* [Integrated](https://signal.org/blog/whatsapp-complete/) into [WhatsApp](https://faq.whatsapp.com/820124435853543) since 2016
* Also used by *Wire, Facebook Messenger, Silent Circle, Pond, XMPP via OMEMO, ChatSecure, ...*

| Messanger           | Installations | Estimated Accounts |
|---------------------|---------------|-----------------|
|  WhatsApp           | 9.64 Billion  | ~~2 Billion [[ref](https://www.whatsapp.com/about/).]~~ 3.5 Billion [[ref](https://github.com/sbaresearch/whatsapp-census).] |
| Facebook Messenger  | 5.89 Billion  | ? |
| ...                 |               |   | 
| Signal              | 136  Million  | ? | 


## Signal Protocol Design Goals
Aims to provide end-to-end encrypted (E2EE) conversations for asynchronous (mobile) messaging, as well as a series of other (informally stated) security properties, such as the following three:

1. **Perfect Forward Secrecy (PFS)**: After a session has been established, this property ensures that an attacker that compromises one device (and thus has access to the *long- and medium-term* secrets) is not able to learn anything about the keys derived in previous sessions. In other words, the attacker is not able to decrypt previously sent messages he might have recorded in transit.

+ This property is achieved by the use of Diffi-Hellman (DH) key agreement at various places in the protocol. 

2. **Post Compromise Security (PCS)**: This property is also referred to as *healing* or *future secrecy* and was first formally described in [[CCG](https://eprint.iacr.org/2016/221)]. It refers to the property that even if an attacker is able to compromise one device during a conversation for a limited amount of time, future conversations might still regain some protection. 

+ Additional entropy is introduced by the ephemeral DH keys received with every message from the other peer, to provide some form of self-healing property. This requires the attacker to sustain his access to continue to be able to read messages in this session. 

3. **(Offline) Deniability**: Refers to the ability that a party can deny having participated in a particular communication session in retrospect. Assuming a communication session between Alice and Bob. Given a transcript of the communication it should not be possible for Bob to convince a third party that Alice has participated knowingly in the session.

+ Signal achieves offline deniability due to the fact that it is implicitly authenticated and uses authenticated encryption instead of signatures [[VGIK](https://eprint.iacr.org/2021/642)].

## Signal Protocol Overview

The Signal Protocol can be further devided into the following parts:
* [X3DH](https://signal.org/docs/specifications/x3dh/) (extended triple Diffie-Hellman) key agreement
    + since late 2023 replaced by [PQXDH](https://signal.org/docs/specifications/pqxdh/), which is coceptually close but adds an additional post-quantum secure key encapsulation mechanism (KEM) into the key derivation function (KDF) 
* [Double ratchet](https://signal.org/docs/specifications/doubleratchet/) algorithm
* [Sesam](https://signal.org/docs/specifications/sesame/) session management for multi-device support, e.g., phone and desktop
* Group chats and [private groups](https://signal.org/blog/signal-private-group-system/) where the server has no record of group membership
* The underlying library [libsignal](https://github.com/signalapp/libsignal) is written in rust and can be found on github

### Signal Protocol Overview

It uses the following cryptographic primitives:
* *unkeyed primitives:*
    + SHA256 hashfunction
* *symmetric primitives:*
    + HMAC-SHA256
    + HKDF with SHA256
    + Authenticated encryption with associated data (AEAD) using AES256 in CBC mode with PKCS#5 padding and HMAC-SHA256 as MAC
* *asymmetric primitives:*
    + DH over curve X25519, or curve X448
    + Ed25519 as signature scheme
    + ML-KEM from [FIPS-203](https://csrc.nist.gov/pubs/fips/203/final) (variant of CRYSTALS-Kyber PQ-KEM) 
    + Keyed-Verification Anonymous Credential [(KVAC)](https://eprint.iacr.org/2019/1416.pdf)

The Signal protocol, as also conceptually [adopted](https://signal.org/blog/whatsapp-complete/) by [WhatsApp](https://faq.whatsapp.com/820124435853543), itself is based on [X3DH](https://signal.org/docs/specifications/x3dh/) (extended triple Diffie-Hellman) key agreement and the [double ratchet](https://signal.org/docs/specifications/doubleratchet/) algorithm. For an in-depth overview of the Signal protocol and some of the desired security properties we refer to [[CCG](https://eprint.iacr.org/2016/221),[VGIK](https://eprint.iacr.org/2021/642),[BFGJS](https://eprint.iacr.org/2021/769)]

## [X3DH](https://signal.org/docs/specifications/x3dh/) and the [Double ratchet](https://signal.org/docs/specifications/doubleratchet/)

<p style="text-align:center;font-size:10px">
<a href="https://math.preview.excalidraw.com/#json=g-0eNcKufOdQuu3is_ytb,NcQ1dkk8LmUp1rYeRrmkkg">(excalidraw)</a>
<a href="./figures/signal-2024-12-17-1108.svg">(image)</a>
</p>
<p style="text-align:center">
<img src="./figures/signal-2024-12-17-1108.png" alt="number systems">
</p>



Since late 2023 Signal replaced X3DH by [PQXDH](https://signal.org/docs/specifications/pqxdh/) and started a process to use PQXDH for new sessions if supported by both peers. On a high level, PQ3DH is comparable to X3DH with the difference that an additional key from a [CRYSTALS-Kyber](https://pq-crystals.org/kyber/) key encapsulation mechanism (KEM) is feed into the key derivation function (KDF) creating the internally used *root*, *chain* and *message* keys. As a consequence, an attacker that recorded the initial handshake and later gets access to a cryptographically relevant quantum computer has to break not only X25519 but also CRYSTALS-Kyber, which is conjectured to be secure also against a quantum computer. This threat is sometimes also referred to as harvest now, decrypt later (HNDL).
For more information see the NIST post-quantum cryptography project, where a variant of CRYSTALS-Kyber is one of the selected finalists and referred to as ML-KEM in [FIPS-203](https://csrc.nist.gov/pubs/fips/203/final). 

## [Sesam](https://signal.org/docs/specifications/sesame/) for Multi Device Support

Modern messengers thrive to support the usage of multiple devices by the same user, e.g., a mobile app and a desktop client. 

There are two basic approaches for supporting multi device E2EE, which have been termed the *leader based* approach and *client fan-out* (also sometimes referred to as *multiplication based approach*) [[GGMJHFU](https://arxiv.org/pdf/2411.11194v2)]:

1. **leader based:**
    The *primary device* -- typically a smart phone -- acts as a leader and serves as the EE2E endpoint pretending
    that there is only a single device per user. 
    The leader device then distributes the messages, also in an encrypted way, to the other devices of the user, 
    also referred to as *companion devices* [[B](https://www.usenix.org/conference/woot24/presentation/beery)]. 

* The drawback is that the leader device represents a single point of failure. If it is offline, no communication with any of the associated devices is possible. WhatsApp followed this approach until mid 2021. 

2. **client fan-out:**
    All devices of the user maintain their own key pair. 
    For messaging, the sender creates an individual E2EE channel with *each* device of the receiver as if messaging multiple recipients [[Sesam](https://signal.org/docs/specifications/sesame/)]. 
  
<p style="text-align:center">
<img src="./figures/multidevice.png" alt="number systems" width="350">
</p>
<p style="text-align:center;font-size:10px">
<a href="https://arxiv.org/pdf/2411.11194v2">(image source)</a>
</p>

  For remaining consistency among all devices of the sender,
    the sending device forwards the message (and other information) to the other devices via encrypted channels as described in the Sesame protocol [[Sesam](https://signal.org/docs/specifications/sesame/)].
    This approach avoids the single point of failure inherent in leader-based method. 
    However, assigning multiple keys to an account reveals the number of devices a user has as other accounts retrieve these keys from a central inventory. 
    Since each device has its own unique key, the recipient can also infer which of the sender’s devices originated the message [[B](https://www.usenix.org/conference/woot24/presentation/beery)].

## Digital Markets Act (DMA)

> The [Digital Markets Act (DMA)](https://eur-lex.europa.eu/eli/reg/2022/1925/oj) aims to ensure a higher degree of competition in European digital markets by preventing large companies from abusing their market power and by allowing new players to enter the market. This regulation targets the largest digital platforms operating in the European Union. They are also known as "gatekeepers" ... ([soruce Wikipedia](https://en.wikipedia.org/wiki/Digital_Markets_Act))

*WhatsApp* has been defined as a *gatekeeper* according to the DMA.
Any gatekeeper has to comply with certain [obligations](https://www.eu-digital-markets-act.com/Digital_Markets_Act_Article_7.html):

  

>The gatekeeper shall make at least the following basic functionalities ... interoperable where the gatekeeper itself provides those functionalities to its own end users:
> ...
> (i) end-to-end text messaging between two individual end users;
> (ii) sharing of images, voice messages, videos and other attached files in end to end communication between two individual end users;

> (b) within 2 years from the designation:
> (i) end-to-end text messaging within groups of individual end users;
> (ii) sharing of images, voice messages, videos and other attached files in end-to-end communication between a group chat and an individual end user;

> (c) within 4 years from the designation:
> (i) end-to-end voice calls between two individual end users;
> (ii) end-to-end video calls between two individual end users;
> (iii) end-to-end voice calls between a group chat and an individual end user;
> (iv) end-to-end video calls between a group chat and an individual end user.

What about E2EE: 
> The level of security, including the **end-to-end encryption**, where applicable, that the gatekeeper provides to its own end users **shall be preserved across the interoperable services**.

# EOF